<!--NAVIGATION-->
<[Week-03-03-Data-Visualization-02](Week-04-01-Data-Preprocessin-01.ipynb) | [Contents](Index.ipynb) | [Week-04-02-Data-Preprocessing-02](Week-04-02-Data-Preprocessing-02.ipynb)  >




# Data Preprocessing 01

- Handling Missing Values
- Aggregate
- Pivot

# Handling Missing Data

Background
- Tutorial data: clean and homoeneous
- Real world data: dirty (contain noise) and complex

Questions:
- How can we preprocess the "not-ready" data?
- How do Pandas work for preprocessing the data in Python?


This Section focuses on the methods on handling missing values
- What kind of missing values are common in the real world data?
- How can we handle the missing values?
- How do Pandas work for handling the missing values?

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous.
In particular, many interesting datasets will have some amount of data missing.
To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python.
Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

## Missing Data Conventions

Missing data is a common problem on data analysis. The presence of missing data in a table or DataFrame requires some strategies. There are at least three strategies:
- Using a _mask_ that globally indicates missing values (using a _mask_)
- Choose a sentinel value that indicates a missing entry (using a sentinel value)
- Use learning approach to replace the missing values (using learning approach)

#### Using a _mask_
- It may involve appropriate values to indicate the null status of a value (e.g., true/false, 0/1,)

#### Using a sentinel value
It may indicate the missing value with some data-specific conventions. 
- Missing integer can be replaced with -9999 (or some rare bit patterns, e.g. 987654321)
- Missing values can be replaced with global convention (using NaN (Not a Number) or a specific value which is a part of the IEEE floating point specification)

#### Using learning approach
It may replace the missing value with some machine learning approach.
- Missing integer can be replaced with a regression prediction result
- Missing nominal value can be replaced with a decision tree classification result

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame.
Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinel value could be some data-specific convention, such as indicating a missing integer value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and systems use different conventions.
For example, the R language uses reserved bit patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

#### How Pandas handling missing values

- There is no such built-in notion of NA values for non-floating point data types (in NumPy)
- However, Pandas will use sentinel values for missing data:
    - ```NaN``` for special floating value
    - ```None``` for object (categorical) value

## Missing Data in Pandas

The way in which Pandas handles missing values is constrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each individual data type to indicate nullness, but this approach turns out to be rather unwieldy.
While R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness, and endianness of the encoding.
Reserving a specific bit pattern in all available NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types (such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of values it can represent.

NumPy does have support for masked arrays – that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad."
Pandas could have derived from this, but the overhead in both storage, computation, and code maintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point ``NaN`` value, and the Python ``None`` object.
This choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interest.

### ``None``: Pythonic missing data (commonly for categorical)

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [3]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between an integer and ``None`` is undefined.

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [4]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [5]:
vals2

array([ 1., nan,  3.,  4.])

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that will ignore these missing values:

In [12]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

In [5]:
data_miss = pd.read_csv("data/name_with_missing.csv")

In [6]:
data_miss

,Name,Weight
0,Amy,65.0
1,Bob,NaN
2,NaN,75.0
3,Daniel,68.0
4,Esther,NaN


### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [8]:
data_miss.isnull()

,Name,Weight
0,False,False
1,False,True
2,True,False
3,False,False
4,False,True


In [9]:
data_miss.notnull()

,Name,Weight
0,True,True
1,True,False
2,False,True
3,True,True
4,True,False


The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). 
the result is straightforward:

In [10]:
data_miss.dropna()

,Name,Weight
0,Amy,65.0
3,Daniel,68.0


In [11]:
data_miss.dropna(axis='columns')

""
0
1
2
3
4


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [41]:
#create a new column and let the values are nan for all rows
data_miss["new_column"] = np.nan
data_miss

,Name,Weight,new_column
0,Amy,65.0,NaN
1,Bob,NaN,NaN
2,NaN,75.0,NaN
3,Daniel,68.0,NaN
4,Esther,NaN,NaN


In [42]:
#delete the columns which all values are nan
data_miss.dropna(axis='columns', how='all')

,Name,Weight
0,Amy,65.0
1,Bob,NaN
2,NaN,75.0
3,Daniel,68.0
4,Esther,NaN


In [12]:
data_miss.dropna(axis='rows')
#delete the rows which has NaN

,Name,Weight
0,Amy,65.0
3,Daniel,68.0


In [27]:
#delete the columns which the non-null values to be kept is minimum as (at least) 4
data_miss.dropna(axis='columns', thresh=4)

,Name
0,Amy
1,Bob
2,NaN
3,Daniel
4,Esther


In [28]:
#delete the rows which the non-null values to be kept is minimum as (at least) 2
#what is the result when you change the thresh = 1?
data_miss.dropna(axis='rows', thresh=2)

,Name,Weight
0,Amy,65.0
3,Daniel,68.0


## Learning Check

1. What is the difference between NaN and np.nan?
2. Is there any difference between .sum() and .nansum()?
3. Mention the difference between .isnull() and .notnull()!
4. How will you delete the nan value from your data?

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [43]:
data_miss

,Name,Weight,new_column
0,Amy,65.0,NaN
1,Bob,NaN,NaN
2,NaN,75.0,NaN
3,Daniel,68.0,NaN
4,Esther,NaN,NaN


We can fill NA entries with a single value, such as zero:

In [44]:
data_miss["Weight"].fillna(0)

0    65.0
1     0.0
2    75.0
3    68.0
4     0.0
Name: Weight, dtype: float64

In [45]:
data_miss

,Name,Weight,new_column
0,Amy,65.0,NaN
1,Bob,NaN,NaN
2,NaN,75.0,NaN
3,Daniel,68.0,NaN
4,Esther,NaN,NaN


We can specify a forward-fill to propagate the previous value forward:

In [46]:
# forward-fill
data_miss.fillna(method='ffill')
#the NaN data will be filled with the previous data

,Name,Weight,new_column
0,Amy,65.0,NaN
1,Bob,65.0,NaN
2,Bob,75.0,NaN
3,Daniel,68.0,NaN
4,Esther,68.0,NaN


Or we can specify a back-fill to propagate the next values backward:

In [47]:
# back-fill
data_miss.fillna(method='bfill')

,Name,Weight,new_column
0,Amy,65.0,NaN
1,Bob,75.0,NaN
2,Daniel,75.0,NaN
3,Daniel,68.0,NaN
4,Esther,NaN,NaN


Notice that if a previous value is not available during a forward fill, the NA value remains.

# Example 1

In this example, there are at least 3 types of missing values.
- Standard Missing values (NaN)
- Non-Standard Missing values
- Unexpected Missing values

In [35]:
data = pd.read_csv('data/property_data.csv')

In [36]:
data
#PID : Property ID
#ST_Num: Street Number
#ST_Name: Street Name
#Own_Occupied : Owner occupy the room (Y/N)
#Num_bedrooms : number of bedroom in the property
#Num_bath : number of bathroom
#Sq_Ft: Square feet

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800
6,100007000.0,NaN,WASHINGTON,NaN,2,HURLEY,950
7,100008000.0,213.0,TREMONT,Y,--,1,NaN
8,100009000.0,215.0,TREMONT,Y,na,2,1800


In [37]:
data.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600


Standard missing values

In [38]:
print(data['ST_NUM'])

0    104.0
1    197.0
2      NaN
3    201.0
4    203.0
5    207.0
6      NaN
7    213.0
8    215.0
Name: ST_NUM, dtype: float64


In [39]:
print(data['ST_NUM'].isnull())

0    False
1    False
2     True
3    False
4    False
5    False
6     True
7    False
8    False
Name: ST_NUM, dtype: bool


Non-Standard Missing values

In [40]:
print(data['NUM_BEDROOMS'])

0      3
1      3
2    NaN
3      1
4      3
5    NaN
6      2
7     --
8     na
Name: NUM_BEDROOMS, dtype: object


In [41]:
print(data['NUM_BEDROOMS'].isnull())

0    False
1    False
2     True
3    False
4    False
5     True
6    False
7    False
8    False
Name: NUM_BEDROOMS, dtype: bool


Pandas recognized the “NA” as a missing value. Unfortunately, the other types weren’t recognized.
If there’s multiple users manually entering data, then this is a common problem. Maybe i like to use “n/a” but you like to use “na”.
An easy way to detect these various formats is to put them in a list. Then when we import the data, Pandas will recognize them right away. Here’s an example of how we would do that.

In [42]:
# Making a list of missing value types
missing_values = ["n/a", "na", "--"]
data1 = pd.read_csv("data/property_data.csv", na_values = missing_values)

In [43]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850.0
3,100004000.0,201.0,BERKELEY,12,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800.0
6,100007000.0,NaN,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,NaN,1,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2,1800.0


In [44]:
print(data1['NUM_BEDROOMS'])

0    3.0
1    3.0
2    NaN
3    1.0
4    3.0
5    NaN
6    2.0
7    NaN
8    NaN
Name: NUM_BEDROOMS, dtype: float64


In [45]:
print(data1['NUM_BEDROOMS'].isnull())

0    False
1    False
2     True
3    False
4    False
5     True
6    False
7     True
8     True
Name: NUM_BEDROOMS, dtype: bool


Unexpected Missing values

In [46]:
print(data1['OWN_OCCUPIED'])

0      Y
1      N
2      N
3     12
4      Y
5      Y
6    NaN
7      Y
8      Y
Name: OWN_OCCUPIED, dtype: object


In [47]:
print(data1['OWN_OCCUPIED'].isnull())

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7    False
8    False
Name: OWN_OCCUPIED, dtype: bool


In the fourth row, there’s the number 12. The response for Owner Occupied should clearly be a string (Y or N), so this numeric type should be a missing value.

To understand the next cell, we will learn a basic concept of `try except` below.

In [49]:
while True:
    x = int(input("Please enter a number: "))
    break

Please enter a number:  a


ValueError: invalid literal for int() with base 10: 'a'

The error appears when the input is not available. In order to avoid the program stop, we can use `try except`.

In [51]:
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:
         print("Oops!  That was no valid number.  Try again...")

Please enter a number:  4


In [52]:
#an example of try except
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
        print('Handling run-time error:', err)

Handling run-time error: division by zero


This example is a little more complicated so we’ll need to think through a strategy for detecting these types of missing values. There’s a number of different approaches, but here’s the way that I’m going to work through this one.

In [53]:
# Detecting numbers 
# continue (for-loop), pass (try-except)
cnt=0
for row in data1['OWN_OCCUPIED']:
    try:
        int(row)
        data1.loc[cnt, 'OWN_OCCUPIED']=np.nan
    except ValueError:
        pass
    cnt+=1

In [54]:
print(data1['OWN_OCCUPIED'])

0      Y
1      N
2      N
3    NaN
4      Y
5      Y
6    NaN
7      Y
8      Y
Name: OWN_OCCUPIED, dtype: object


In [55]:
print(data1['OWN_OCCUPIED'].isnull())

0    False
1    False
2    False
3     True
4    False
5    False
6     True
7    False
8    False
Name: OWN_OCCUPIED, dtype: bool


Summarizing Missing Values

In [56]:
# Total missing values for each feature
print(data1.isnull().sum())

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    2
NUM_BEDROOMS    4
NUM_BATH        1
SQ_FT           2
dtype: int64


In [57]:
#Any missing values
print(data1.isnull().values.any())

True


In [58]:
# Total number of missing values
print(data1.isnull().sum().sum())

12


In [59]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800.0
6,100007000.0,NaN,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,NaN,1,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2,1800.0


In [60]:
# Replace missing values with a number
data1['ST_NUM'].fillna(125, inplace=True)

In [61]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,125.0,LEXINGTON,N,NaN,1,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800.0
6,100007000.0,125.0,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,NaN,1,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2,1800.0


In [62]:
#replace a location-based imputation
data1.loc[2, 'ST_NUM'] = 130

In [63]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,130.0,LEXINGTON,N,NaN,1,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800.0
6,100007000.0,125.0,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,NaN,1,NaN
8,100009000.0,215.0,TREMONT,Y,NaN,2,1800.0


A very common way to replace missing values is using a median.

In [64]:
median = data1['NUM_BEDROOMS'].median()

In [65]:
median

3.0

In [66]:
#Replace using median
median = data1['NUM_BEDROOMS'].median()
data1['NUM_BEDROOMS'].fillna(median, inplace=True)

In [67]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,130.0,LEXINGTON,N,3.0,1,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,3.0,1,800.0
6,100007000.0,125.0,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,3.0,1,NaN
8,100009000.0,215.0,TREMONT,Y,3.0,2,1800.0


In [68]:
#Replace Nan SQ_FT using mean
b_mean = data1['SQ_FT'].mean()
data1['SQ_FT'].fillna(b_mean, inplace=True)

In [69]:
data1

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,1100.0
2,100003000.0,130.0,LEXINGTON,N,3.0,1,850.0
3,100004000.0,201.0,BERKELEY,NaN,1.0,NaN,700.0
4,NaN,203.0,BERKELEY,Y,3.0,2,1600.0
5,100006000.0,207.0,BERKELEY,Y,3.0,1,800.0
6,100007000.0,125.0,WASHINGTON,NaN,2.0,HURLEY,950.0
7,100008000.0,213.0,TREMONT,Y,3.0,1,1100.0
8,100009000.0,215.0,TREMONT,Y,3.0,2,1800.0


In [70]:
#if you are going to drop the missing values
data1.dropna()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,1100.0
2,100003000.0,130.0,LEXINGTON,N,3.0,1,850.0
5,100006000.0,207.0,BERKELEY,Y,3.0,1,800.0
7,100008000.0,213.0,TREMONT,Y,3.0,1,1100.0
8,100009000.0,215.0,TREMONT,Y,3.0,2,1800.0


## Summary

In this section, we learn about:
- What is Missing Value.
- How to handle missing value
- Some methods to handle missing value with an example